In [10]:
datapath = "/home/wbm001/deeplpi/DeepLPI/data/"
datafile = '/home/wbm001/deeplpi/DeepLPI/data/mol_dict.csv'
modelpath = "/home/wbm001/deeplpi/DeepLPI/mol2vec/model_300dim.pkl"

In [2]:
from rdkit.Chem import SDMolSupplier
from rdkit.Chem.Draw import IPythonConsole

import pandas as pd

data = pd.read_csv(datafile)

In [3]:
data = data[["0"]]
data

,0
0,Cc1c(cnn1CC1(CCCC(C)(C)C1)N1CCOCC1)-c1ccc(nc1C...
1,NCCSc1ccccc1-c1cnc(N)c(n1)-c1nnc(o1)-c1ccccc1
2,CCCCCOC(=O)N1CCN(CC1)C(=O)[C@H](CCC(O)=O)NC(=O...
3,Nc1ccc2nc(CCCCCCCCc3nc4ccc(N)cc4[nH]3)[nH]c2c1
4,Cc1nc2CCCCn2c(=O)c1CCN(Cc1ccc2OCOc2c1)c1nc(ncc...
...,...
230328,CSC(=S)NCCc1ccc(F)cc1
230329,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](C)C(=...
230330,CN1CCN(CC1)c1ccc(cc1NC(=O)c1c[nH]c(=O)cc1C(F)(...
230331,OC(=O)c1nc2n(n1)c1c(Cl)cc(Cl)cc1[nH]c2=O


In [4]:
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from gensim.models import Word2Vec
from rdkit.Chem import MolFromSmiles as sm
import numpy as np

data["sent"] = data["0"].map(lambda x : sm(x))

RDKit ERROR: [05:27:31] Explicit valence for atom # 2 N, 4, is greater than permitted
RDKit ERROR: [05:27:32] Can't kekulize mol.  Unkekulized atoms: 10 12 13 14 15
RDKit ERROR: 
RDKit ERROR: [05:27:32] Explicit valence for atom # 1 B, 6, is greater than permitted
RDKit ERROR: [05:27:32] Can't kekulize mol.  Unkekulized atoms: 20 21 22 23 24
RDKit ERROR: 
RDKit ERROR: [05:27:32] Can't kekulize mol.  Unkekulized atoms: 2 3 23
RDKit ERROR: 
RDKit ERROR: [05:27:33] Can't kekulize mol.  Unkekulized atoms: 22 23 24 25 26
RDKit ERROR: 
RDKit ERROR: [05:27:33] Explicit valence for atom # 31 H, 2, is greater than permitted
RDKit ERROR: [05:27:33] Explicit valence for atom # 24 N, 4, is greater than permitted
RDKit ERROR: [05:27:33] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6 7 8 15 16
RDKit ERROR: 
RDKit ERROR: [05:27:34] Explicit valence for atom # 22 N, 4, is greater than permitted
RDKit ERROR: [05:27:34] Explicit valence for atom # 31 N, 4, is greater than permitted
RDKit ERROR: [05:2

In [5]:
def mol2alt(x):
    try:
        re = mol2alt_sentence(x,1)
    except:
        re = None
    return re

In [15]:
data

,0,sent
0,Cc1c(cnn1CC1(CCCC(C)(C)C1)N1CCOCC1)-c1ccc(nc1C...,"<img data-content=""rdkit/molecule"" src=""data:i..."
1,NCCSc1ccccc1-c1cnc(N)c(n1)-c1nnc(o1)-c1ccccc1,"<img data-content=""rdkit/molecule"" src=""data:i..."
2,CCCCCOC(=O)N1CCN(CC1)C(=O)[C@H](CCC(O)=O)NC(=O...,"<img data-content=""rdkit/molecule"" src=""data:i..."
3,Nc1ccc2nc(CCCCCCCCc3nc4ccc(N)cc4[nH]3)[nH]c2c1,"<img data-content=""rdkit/molecule"" src=""data:i..."
4,Cc1nc2CCCCn2c(=O)c1CCN(Cc1ccc2OCOc2c1)c1nc(ncc...,"<img data-content=""rdkit/molecule"" src=""data:i..."
...,...,...
230328,CSC(=S)NCCc1ccc(F)cc1,"<img data-content=""rdkit/molecule"" src=""data:i..."
230329,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](C)C(=...,"<img data-content=""rdkit/molecule"" src=""data:i..."
230330,CN1CCN(CC1)c1ccc(cc1NC(=O)c1c[nH]c(=O)cc1C(F)(...,"<img data-content=""rdkit/molecule"" src=""data:i..."
230331,OC(=O)c1nc2n(n1)c1c(Cl)cc(Cl)cc1[nH]c2=O,"<img data-content=""rdkit/molecule"" src=""data:i..."


In [13]:
data_sentence = data["sent"].map(lambda x : mol2alt(x))

In [17]:
data["alt_sent"] = data_sentence
data

,0,sent,alt_sent
0,Cc1c(cnn1CC1(CCCC(C)(C)C1)N1CCOCC1)-c1ccc(nc1C...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 422715066, 3217380708, 3286720230..."
1,NCCSc1ccccc1-c1cnc(N)c(n1)-c1nnc(o1)-c1ccccc1,"<img data-content=""rdkit/molecule"" src=""data:i...","[847957139, 2592785365, 2245384272, 787341104,..."
2,CCCCCOC(=O)N1CCN(CC1)C(=O)[C@H](CCC(O)=O)NC(=O...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 3542456614, 2245384272, 117312591..."
3,Nc1ccc2nc(CCCCCCCCc3nc4ccc(N)cc4[nH]3)[nH]c2c1,"<img data-content=""rdkit/molecule"" src=""data:i...","[847957139, 1083852209, 3217380708, 3975295864..."
4,Cc1nc2CCCCn2c(=O)c1CCN(Cc1ccc2OCOc2c1)c1nc(ncc...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 422715066, 3217380708, 4033380444..."
...,...,...,...
230328,CSC(=S)NCCc1ccc(F)cc1,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 2592252298, 1026928756, 401661240..."
230329,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](C)C(=...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 3537119515, 2245273601, 300433380..."
230330,CN1CCN(CC1)c1ccc(cc1NC(=O)c1c[nH]c(=O)cc1C(F)(...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 3657471097, 2092489639, 202181243..."
230331,OC(=O)c1nc2n(n1)c1c(Cl)cc(Cl)cc1[nH]c2=O,"<img data-content=""rdkit/molecule"" src=""data:i...","[864662311, 1533864325, 2246699815, 2784506312..."


In [18]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load(modelpath)

In [21]:
data = data.dropna()
data

,0,sent,alt_sent
0,Cc1c(cnn1CC1(CCCC(C)(C)C1)N1CCOCC1)-c1ccc(nc1C...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 422715066, 3217380708, 3286720230..."
1,NCCSc1ccccc1-c1cnc(N)c(n1)-c1nnc(o1)-c1ccccc1,"<img data-content=""rdkit/molecule"" src=""data:i...","[847957139, 2592785365, 2245384272, 787341104,..."
2,CCCCCOC(=O)N1CCN(CC1)C(=O)[C@H](CCC(O)=O)NC(=O...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 3542456614, 2245384272, 117312591..."
3,Nc1ccc2nc(CCCCCCCCc3nc4ccc(N)cc4[nH]3)[nH]c2c1,"<img data-content=""rdkit/molecule"" src=""data:i...","[847957139, 1083852209, 3217380708, 3975295864..."
4,Cc1nc2CCCCn2c(=O)c1CCN(Cc1ccc2OCOc2c1)c1nc(ncc...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 422715066, 3217380708, 4033380444..."
...,...,...,...
230328,CSC(=S)NCCc1ccc(F)cc1,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 2592252298, 1026928756, 401661240..."
230329,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](C)C(=...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 3537119515, 2245273601, 300433380..."
230330,CN1CCN(CC1)c1ccc(cc1NC(=O)c1c[nH]c(=O)cc1C(F)(...,"<img data-content=""rdkit/molecule"" src=""data:i...","[2246728737, 3657471097, 2092489639, 202181243..."
230331,OC(=O)c1nc2n(n1)c1c(Cl)cc(Cl)cc1[nH]c2=O,"<img data-content=""rdkit/molecule"" src=""data:i...","[864662311, 1533864325, 2246699815, 2784506312..."


In [39]:
molvec = sentences2vec(data["alt_sent"],model,unseen='UNK')
molvec

array([[ 6.40300894e+00, -8.65941143e+00, -3.88281345e+00, ...,
        -8.35119343e+00, -2.02949200e+01, -3.15997910e+00],
       [ 3.00149393e+00, -3.29223132e+00, -3.70306897e+00, ...,
        -3.24070477e+00, -1.14652309e+01, -1.77243710e-01],
       [ 1.38151288e+00, -5.13364792e+00, -4.28539801e+00, ...,
        -1.24435253e+01, -2.00695038e+01, -7.50611210e+00],
       ...,
       [ 5.78267097e+00, -7.28869343e+00, -6.20715284e+00, ...,
        -6.37059498e+00, -1.62003536e+01, -1.41879189e+00],
       [ 3.69474506e+00, -2.39012504e+00, -3.23361468e+00, ...,
        -1.89487052e+00, -8.07069397e+00, -2.26348495e+00],
       [ 1.01475574e-01,  2.35239184e-03, -1.41808093e-02, ...,
         5.98901361e-02, -5.97242266e-02,  2.44590491e-02]], dtype=float32)

In [55]:
molembed = np.hstack((np.array(data.index).reshape(-1,1) , molvec))

In [63]:
molembeddf = pd.DataFrame(molembed)
molembeddf[0] = molembeddf[0].astype(int)
molembeddf = molembeddf.set_index(0)
molembeddf

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
0,6.403009,-8.659411,-3.882813,7.825369,-2.434460,-1.051279,-21.530718,2.214782,9.988102,5.409351,...,0.317064,23.724264,14.760700,-5.297044,-18.148996,-7.443026,-12.752075,-8.351193,-20.294920,-3.159979
1,3.001494,-3.292231,-3.703069,10.514475,0.124146,2.700886,-13.521187,-1.278113,6.229233,-0.598048,...,-2.850580,10.577876,14.544487,-2.212488,-5.906467,-2.570442,-4.611706,-3.240705,-11.465231,-0.177244
2,1.381513,-5.133648,-4.285398,1.358949,5.510566,-0.769088,-21.339130,-3.075223,12.862975,0.782963,...,-0.264127,17.215847,17.731188,3.055171,-14.980936,-7.293749,-6.002640,-12.443525,-20.069504,-7.506112
3,1.197093,-2.943224,-3.440884,8.910519,2.699831,2.631797,-15.425988,-2.248305,6.479279,-4.111322,...,-1.781228,10.832394,15.226110,0.804459,-6.303400,-4.914831,-3.314373,-5.156674,-13.546863,-1.997584
4,5.578038,-6.557280,-4.329960,8.205982,0.267682,1.021811,-14.067959,-0.273367,6.118787,3.290239,...,-3.025323,12.568911,10.325743,-3.579992,-12.897421,-5.325835,-9.812045,-6.258411,-12.949100,-2.547693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230328,-0.257810,-1.050426,0.409205,4.003800,0.185199,-0.168164,-7.843135,-2.971186,7.174545,1.910118,...,-0.575556,3.331519,6.885682,2.034392,-0.840564,-0.681459,-4.204550,-2.199346,-6.420996,-0.839175
230329,-2.769332,-2.736311,-5.915005,3.897100,5.776762,-3.660158,-20.984577,-5.446694,20.057419,1.138266,...,-1.299875,12.609625,17.236053,7.737417,-12.149308,-5.570494,0.542005,-14.704156,-16.932697,-2.908373
230330,5.782671,-7.288693,-6.207153,8.290014,-3.059343,-2.390173,-14.741295,0.373247,11.279165,5.580946,...,-1.526583,16.096859,14.109312,-1.494534,-11.881902,-3.367565,-6.204700,-6.370595,-16.200354,-1.418792


In [ ]:
molembeddf.to_csv(datapath+"mol_embed.csv")